# Exercises


##  Pandas

**MovieLens database**

In [1]:
import pandas as pd

users_data =  '../ml-1m/users.dat'
ratings_data = '../ml-1m/ratings.dat'
movies_data =  '../ml-1m/movies.dat'


unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv(users_data, sep='::', header=None, names=unames, engine='python',encoding="utf")
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_data, sep='::', header=None, names=rnames,  engine='python',encoding="utf")
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv(movies_data, sep='::', header=None, names=mnames,  engine='python',encoding="latin1")

data = pd.merge(pd.merge(ratings, users), movies)
print (data[:30])


    user_id  movie_id  rating   timestamp gender  age  occupation    zip  \
0         1      1193       5   978300760      F    1          10  48067   
1         2      1193       5   978298413      M   56          16  70072   
2        12      1193       4   978220179      M   25          12  32793   
3        15      1193       4   978199279      M   25           7  22903   
4        17      1193       5   978158471      M   50           1  95350   
5        18      1193       4   978156168      F   18           3  95825   
6        19      1193       5   982730936      M    1          10  48073   
7        24      1193       5   978136709      F   25           7  10023   
8        28      1193       3   978125194      F   25           1  14607   
9        33      1193       5   978557765      M   45           3  55421   
10       39      1193       5   978043535      M   18           4  61820   
11       42      1193       3   978038981      M   25           8  24502   
12       44 

1- Filter films that have received at least 250 ratings:

In [6]:
rev_by_movie = ratings.groupby('movie_id')['user_id'].count()
data_filtered = data[data['movie_id'].map(rev_by_movie)>250]
data_filtered.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


**Using the previous filtered films**:

2- Obtain the mean ratings for each movie grouped by gender

In [32]:
### Your Code HERE
rating_by_gender=data_filtered.groupby(['movie_id', 'title','gender']).rating.mean()
rating_by_gender

movie_id  title                       gender
1         Toy Story (1995)            F         4.187817
                                      M         4.130552
2         Jumanji (1995)              F         3.278409
                                      M         3.175238
3         Grumpier Old Men (1995)     F         3.073529
                                                  ...   
3948      Meet the Parents (2000)     M         3.641838
3949      Requiem for a Dream (2000)  F         3.950000
                                      M         4.174107
3952      Contender, The (2000)       F         3.761905
                                      M         3.787986
Name: rating, Length: 2428, dtype: float64

**Using the previous filtered films:**

3- Show the films most valued by women.

In [37]:
### Your Code HERE
# Using the same object as before
rating_by_gender[rating_by_gender.index.map(lambda g: g[-1]=='F')].sort_values(ascending=False)[:10]

# Redoing the object
data_filtered[data_filtered['gender']=='F'].groupby(['movie_id', 'title','gender']).rating.mean().sort_values(ascending=False)[:10]

movie_id  title                                                   gender
745       Close Shave, A (1995)                                   F         4.644444
1148      Wrong Trousers, The (1993)                              F         4.588235
922       Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)           F         4.572650
720       Wallace & Gromit: The Best of Aardman Animation (1996)  F         4.563107
527       Schindler's List (1993)                                 F         4.562602
318       Shawshank Redemption, The (1994)                        F         4.539075
1223      Grand Day Out, A (1992)                                 F         4.537879
1207      To Kill a Mockingbird (1962)                            F         4.536667
3429      Creature Comforts (1990)                                F         4.513889
50        Usual Suspects, The (1995)                              F         4.513317
Name: rating, dtype: float64

**Using the previous filtered films:**

4- Now we wonder which movies are rated more differently between men and women. 
    * Which films have more different rating and are more highly valued by women? 
    * And the films preferred by men which doesn't liked women? 
    * What are the films that have generated the most discordant ratings, regardless of gender?

In [58]:
### Your Code HERE
is_men = data['gender']=='M'
is_women = data['gender']=='F'

rating_by_men = data[is_men].groupby('movie_id').rating.mean()
rating_by_women = data[is_women].groupby('movie_id').rating.mean()
std = data[is_women].groupby('movie_id')['rating'].mean()

movies['men_rating'] = movies['movie_id'].map(rating_by_men)
movies['women_rating'] = movies['movie_id'].map(rating_by_women)
movies['std'] = movies['movie_id'].map(std)

movies['diff_rating'] = movies['women_rating']-movies['men_rating']

print('Which films have more different rating and are more highly valued by women?')
print(movies.sort_values('diff_rating', ascending=False).head())

print('\n\nAnd the films preferred by men which doesn\'t liked women?')
print(movies.sort_values('diff_rating', ascending=True).head())

print("\n\nWhat are the films that have generated the most discordant ratings, regardless of gender?")
print(movies.sort_values('std', ascending=False).head())


Which films have more different rating and are more highly valued by women?
      movie_id                                              title  \
3067      3136                       James Dean Story, The (1957)   
679        687                                Country Life (1994)   
2754      2823  Spiders, The (Die Spinnen, 1. Teil: Der Golden...   
766        776                                   Babyfever (1994)   
3572      3641                           Woman of Paris, A (1923)   

             genres  men_rating  women_rating  diff_rating       std  
3067    Documentary    1.000000      4.000000     3.000000  4.000000  
679   Drama|Romance    2.000000      5.000000     3.000000  5.000000  
2754   Action|Drama    1.000000      4.000000     3.000000  4.000000  
766    Comedy|Drama    1.000000      3.666667     2.666667  3.666667  
3572          Drama    2.428571      5.000000     2.571429  5.000000  


And the films preferred by men which doesn't liked women?
     movie_id          

5- Calculate the average rating of each user. 

In [ ]:
### Your Code HERE

6- What is the highest rated movie in average that have received at least 250 ratings?

In [ ]:
### Your Code HERE

7- Define a function called  <b>top_movies</b> that given an user and a number n, it returns the top n movies for this user.



In [72]:
def top_movies(user, n=10):
### Your Code HERE
    return data[data['user_id']==user].sort_values('rating', ascending=False)[:n]['title']
top_movies(users.iloc[0]['user_id'])

0        One Flew Over the Cuckoo's Nest (1975)
50759                              Dumbo (1941)
41626                          Toy Story (1995)
19503                         Awakenings (1990)
43703                           Rain Man (1988)
25853                   Schindler's List (1993)
37339                         Cinderella (1950)
15859                Sound of Music, The (1965)
28501                         Pocahontas (1995)
49748                       Mary Poppins (1964)
Name: title, dtype: object

**Data from CSV**

8- Read data from csv file: `ma-ba.csv`. Count the number of times `Barça` wins `Madrid` and compute the stadistics of % win, % lose and % draw.

In [ ]:
### Your Code HERE

#### !!! Compare your results with the csv files located in results.zip !!!